# Chess Player Skill Classifier - STREAMING VERSION
## Extract Games WITHOUT Downloading 30GB File!

This version uses **streaming extraction** to get games directly from Lichess without downloading the full 30GB database.

**Key Innovation:**
- ✅ Downloads data in **small chunks** (streams)
- ✅ Extracts games **on-the-fly**
- ✅ Stops automatically when target reached
- ✅ Only downloads ~100-500MB total (not 30GB!)

**Runtime:** ~1-2 hours total

---

## 📦 Step 1: Setup Environment

In [1]:
# Install dependencies
!apt-get update -qq && apt-get install -y stockfish -qq
!pip install -q python-chess numpy pandas scikit-learn matplotlib seaborn joblib tqdm zstandard requests

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Selecting previously unselected package stockfish.
(Reading database ... 117540 files and directories currently installed.)
Preparing to unpack .../stockfish_14.1-1_amd64.deb ...
Unpacking stockfish (14.1-1) ...
Setting up stockfish (14.1-1) ...
Processing triggers for man-db (2.10.2-1) ...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 43.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [2]:
# Imports
import io, os, json, warnings
import chess.pgn
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, accuracy_score, precision_recall_fscore_support
from imblearn.over_sampling import SMOTE
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
from tqdm import tqdm
import zstandard as zstd
import requests

warnings.filterwarnings('ignore')
np.random.seed(42)

print('✓ All dependencies installed')

✓ All dependencies installed


In [3]:
# Mount Google Drive
from google.colab import drive, files

mount_point = '/content/drive'
drive.mount(mount_point, force_remount=True)

DRIVE_DIR = '/content/drive/MyDrive/chess_ml_streams'
os.makedirs(DRIVE_DIR, exist_ok=True)

def dp(filename):
    return os.path.join(DRIVE_DIR, filename)

print(f'✓ Drive mounted → {DRIVE_DIR}')

Mounted at /content/drive
✓ Drive mounted → /content/drive/MyDrive/chess_ml_streams


---
## 📤 Step 2: Upload chess_analyzer.py

In [ ]:
# Upload chess_analyzer.py
if os.path.exists(dp('chess_analyzer.py')):
    import shutil
    shutil.copy(dp('chess_analyzer.py'), '/content/chess_analyzer.py')
    print('✓ chess_analyzer.py loaded from Drive')
else:
    print('📤 Upload your chess_analyzer.py file:')
    uploaded = files.upload()
    if 'chess_analyzer.py' in uploaded:
        with open('/content/chess_analyzer.py', 'wb') as f:
            f.write(uploaded['chess_analyzer.py'])
        import shutil
        shutil.copy('/content/chess_analyzer.py', dp('chess_analyzer.py'))
        print('✓ chess_analyzer.py uploaded')
    else:
        raise FileNotFoundError('chess_analyzer.py required!')

import sys
sys.path.insert(0, '/content')
from chess_analyzer import ChessAnalyzer, extract_features
print('✓ ChessAnalyzer imported')

✓ chess_analyzer.py loaded from Drive
✓ ChessAnalyzer imported


---
## 🌊 Step 3: Stream & Extract Games from Lichess

This is the **magic part** - we download and process games in real-time without saving the 30GB file!

In [ ]:
# Configuration
GAMES_PER_CATEGORY = 3000  # Adjust this (500-5000 recommended)
MIN_MOVES = 20
MAX_MOVES = 120

# Lichess database URL (January 2024)
LICHESS_URL = 'https://database.lichess.org/standard/lichess_db_standard_rated_2024-01.pgn.zst'

# Rating brackets
RATING_BRACKETS = {
    'weak': (1200, 1600),
    'average': (1600, 2000),
    'excellent': (2000, 2400)
}

print(f'🎯 Target: {GAMES_PER_CATEGORY} games per category')
print(f'📊 Total: {GAMES_PER_CATEGORY * 3} games\n')
print('Rating brackets:')
for skill, (min_elo, max_elo) in RATING_BRACKETS.items():
    print(f'  {skill:10s}: {min_elo}-{max_elo} Elo')

🎯 Target: 3000 games per category
📊 Total: 9000 games

Rating brackets:
  weak      : 1200-1600 Elo
  average   : 1600-2000 Elo
  excellent : 2000-2400 Elo


In [ ]:
def stream_and_extract_games(url, games_per_category, min_moves, max_moves):
    """
    Stream Lichess database and extract games WITHOUT downloading full file

    This downloads the file in chunks and processes on-the-fly,
    stopping once we have enough games.
    """
    categorized_games = {skill: [] for skill in RATING_BRACKETS.keys()}

    # Check for existing checkpoint
    checkpoint_file = dp('extraction_checkpoint.json')
    if os.path.exists(checkpoint_file):
        try:
            with open(checkpoint_file) as f:
                categorized_games = json.load(f)
            print(f'\n🔄 Resuming from checkpoint:')
            for skill, games in categorized_games.items():
                print(f'  {skill:10s}: {len(games)} games')
        except:
            print('⚠️ Checkpoint corrupted, starting fresh')
            categorized_games = {skill: [] for skill in RATING_BRACKETS.keys()}

    # Check if already complete
    if all(len(categorized_games[s]) >= games_per_category for s in categorized_games.keys()):
        print('\n✓ Already have enough games!')
        return categorized_games

    print(f'\n🌊 Starting streaming extraction from Lichess...')
    print(f'📡 URL: {url}')
    print(f'⏱️  This will take 30-60 minutes\n')

    # Stream the file
    response = requests.get(url, stream=True)
    total_size = int(response.headers.get('content-length', 0))

    print(f'Full database size: {total_size / (1024**3):.1f} GB')
    print('We\'ll only download what we need!\n')

    # Setup decompressor
    dctx = zstd.ZstdDecompressor()
    stream_reader = dctx.stream_reader(response.raw)
    text_stream = io.TextIOWrapper(stream_reader, encoding='utf-8', errors='ignore')

    total_processed = 0
    total_skipped = 0
    bytes_downloaded = 0

    pbar = tqdm(
        total=games_per_category * 3,
        desc="Extracting games",
        unit="games"
    )

    # Set initial progress
    current_total = sum(len(categorized_games[s]) for s in categorized_games.keys())
    pbar.update(current_total)

    checkpoint_interval = 100  # Save every 100 games
    last_checkpoint = current_total

    try:
        while True:
            # Check if done
            if all(len(categorized_games[s]) >= games_per_category for s in categorized_games.keys()):
                print('\n\n✓ Target reached! Stopping download...')
                break

            try:
                game = chess.pgn.read_game(text_stream)
                if game is None:
                    print('\n\n⚠️ Reached end of database')
                    break

                total_processed += 1

                # Get ratings
                try:
                    white_elo = int(game.headers.get('WhiteElo', 0))
                    black_elo = int(game.headers.get('BlackElo', 0))
                except (ValueError, TypeError):
                    total_skipped += 1
                    continue

                # Filter by move count
                num_moves = len(list(game.mainline_moves()))
                if num_moves < min_moves or num_moves > max_moves:
                    total_skipped += 1
                    continue

                # Check rating brackets
                for skill_level, (min_elo, max_elo) in RATING_BRACKETS.items():
                    if len(categorized_games[skill_level]) >= games_per_category:
                        continue

                    # Both players in range
                    if (min_elo <= white_elo <= max_elo and
                        min_elo <= black_elo <= max_elo):

                        exporter = chess.pgn.StringExporter(
                            headers=True,
                            variations=False,
                            comments=True
                        )
                        pgn_str = game.accept(exporter)

                        categorized_games[skill_level].append({
                            'pgn': pgn_str,
                            'white_elo': white_elo,
                            'black_elo': black_elo,
                            'skill_level': skill_level
                        })

                        pbar.update(1)
                        break

                # Checkpoint
                current_total = sum(len(categorized_games[s]) for s in categorized_games.keys())
                if current_total - last_checkpoint >= checkpoint_interval:
                    with open(checkpoint_file, 'w') as f:
                        json.dump(categorized_games, f)
                    last_checkpoint = current_total

                # Update progress bar postfix every 5000 games
                if total_processed % 5000 == 0:
                    pbar.set_postfix({
                        'weak': len(categorized_games['weak']),
                        'avg': len(categorized_games['average']),
                        'exc': len(categorized_games['excellent'])
                    })

            except Exception as e:
                total_skipped += 1
                continue

    except KeyboardInterrupt:
        print('\n\n⚠️ Interrupted by user. Saving progress...')

    finally:
        pbar.close()
        text_stream.close()
        response.close()

    # Final save
    with open(checkpoint_file, 'w') as f:
        json.dump(categorized_games, f)

    print(f'\n\n✓ Extraction complete!')
    print(f'  Games processed: {total_processed:,}')
    print(f'  Games skipped: {total_skipped:,}')
    print(f'\n📊 Games extracted per category:')
    for skill, games in categorized_games.items():
        print(f'  {skill:10s}: {len(games):,} games')

    # Estimate data downloaded
    estimated_mb = (total_processed / 100000) * 500  # Rough estimate
    print(f'\n💾 Estimated data downloaded: ~{estimated_mb:.0f} MB (not 30 GB!)')

    return categorized_games

# Run streaming extraction
categorized_games = stream_and_extract_games(
    LICHESS_URL,
    GAMES_PER_CATEGORY,
    MIN_MOVES,
    MAX_MOVES
)


🌊 Starting streaming extraction from Lichess...
📡 URL: https://database.lichess.org/standard/lichess_db_standard_rated_2024-01.pgn.zst
⏱️  This will take 30-60 minutes

Full database size: 30.2 GB
We'll only download what we need!



Extracting games: 100%|██████████| 9000/9000 [02:04<00:00, 72.14games/s, weak=3000, avg=3000, exc=2670]



✓ Target reached! Stopping download...




✓ Extraction complete!
  Games processed: 22,481
  Games skipped: 2,156

📊 Games extracted per category:
  weak      : 3,000 games
  average   : 3,000 games
  excellent : 3,000 games

💾 Estimated data downloaded: ~112 MB (not 30 GB!)


In [ ]:
# Save extracted games
with open(dp('categorized_games.json'), 'w') as f:
    json.dump(categorized_games, f)
print('✓ Saved categorized_games.json to Drive')

# Clean up checkpoint
checkpoint_file = dp('extraction_checkpoint.json')
if os.path.exists(checkpoint_file):
    os.remove(checkpoint_file)
    print('✓ Removed checkpoint file')

✓ Saved categorized_games.json to Drive
✓ Removed checkpoint file


---
## ⚙️ Step 4: Analyze Games with Stockfish

**⏱️ This takes 1-2 hours for 6000 games**

In [ ]:
# Load games
if 'categorized_games' not in globals():
    with open(dp('categorized_games.json')) as f:
        categorized_games = json.load(f)
    print('✓ Loaded categorized games')

# Flatten
all_games_list = []
for skill_level, games in categorized_games.items():
    all_games_list.extend(games)

print(f'📊 Total games to analyze: {len(all_games_list):,}')

📊 Total games to analyze: 9,000


In [ ]:
def analyze_games_batch(games_list, batch_size=50):
    """Analyze games with Stockfish"""
    analyzed_games = []

    # Check for partial
    partial_file = dp('game_analyses_partial.json')
    if os.path.exists(partial_file):
        try:
            with open(partial_file) as f:
                analyzed_games = json.load(f)
            print(f'🔄 Resuming from {len(analyzed_games)} games')
        except:
            analyzed_games = []

    start_idx = len(analyzed_games)

    with ChessAnalyzer() as analyzer:
        for i in tqdm(range(start_idx, len(games_list)), desc="Analyzing"):
            game_data = games_list[i]

            try:
                game = chess.pgn.read_game(io.StringIO(game_data['pgn']))
                player_name = game.headers.get('White', 'Player')

                analysis = analyzer.analyze_game(game_data['pgn'], player_name)

                if analysis and analysis.total_moves > 0:
                    analyzed_games.append({
                        'pgn': game_data['pgn'],
                        'white_elo': game_data['white_elo'],
                        'black_elo': game_data['black_elo'],
                        'skill_level': game_data['skill_level'],
                        'player_color': analysis.player_color,
                        'result': analysis.result,
                        'total_moves': analysis.total_moves,
                        'moves': [
                            {
                                'move_number': m.move_number,
                                'move': m.move,
                                'eval_before': m.eval_before,
                                'eval_after': m.eval_after,
                                'centipawn_loss': min(m.centipawn_loss, 500),
                                'best_move': m.best_move,
                                'time_spent': m.time_spent,
                                'clock_remaining': m.clock_remaining,
                                'phase': m.phase,
                                'is_blunder': m.is_blunder,
                                'is_mistake': m.is_mistake,
                                'is_inaccuracy': m.is_inaccuracy,
                                'is_good': m.is_good,
                                'is_best': m.is_best,
                                'is_brilliant': m.is_brilliant
                            }
                            for m in analysis.moves
                        ]
                    })

            except Exception as e:
                continue

            if (i + 1) % batch_size == 0:
                with open(partial_file, 'w') as f:
                    json.dump(analyzed_games, f)

    # Final save
    with open(dp('game_analyses.json'), 'w') as f:
        json.dump(analyzed_games, f)

    if os.path.exists(partial_file):
        os.remove(partial_file)

    print(f'\n✓ Analyzed {len(analyzed_games)} games')
    return analyzed_games

print('🚀 Starting Stockfish analysis...')
analyzed_games = analyze_games_batch(all_games_list)

🚀 Starting Stockfish analysis...
🔄 Resuming from 6400 games


Analyzing:   0%|          | 1/2600 [00:05<3:42:58,  5.15s/it]

⚠️ 3 moves with extreme evals (mate scenarios) - capped at 500cp


Analyzing:   0%|          | 2/2600 [00:15<6:05:18,  8.44s/it]

⚠️ 8 moves with extreme evals (mate scenarios) - capped at 500cp


...

⚠️ 9 moves with extreme evals (mate scenarios) - capped at 500cp


Analyzing: 100%|█████████▉| 2599/2600 [5:24:37<00:06,  6.67s/it]

⚠️ 2 moves with extreme evals (mate scenarios) - capped at 500cp
⚠️ 1 moves with extreme evals (mate scenarios) - capped at 500cp


Analyzing: 100%|██████████| 2600/2600 [5:24:52<00:00,  7.50s/it]



✓ Analyzed 9000 games


---
## 📊 Step 5: Extract Features & Labels

In [9]:
# Load if needed
if 'analyzed_games' not in globals():
    with open(dp('/content/drive/MyDrive/chess_ml_streams/game_analyses.json')) as f:
        analyzed_games = json.load(f)

print(f'📊 Extracting features from {len(analyzed_games)} games...')

📊 Extracting features from 9000 games...


In [10]:
from types import SimpleNamespace

all_features = []

for game_data in tqdm(analyzed_games, desc="Features"):
    try:
        moves = [SimpleNamespace(**m) for m in game_data['moves']]
        game_analysis = SimpleNamespace(
            player_color=game_data['player_color'],
            result=game_data['result'],
            moves=moves,
            total_moves=game_data['total_moves']
        )

        features = extract_features([game_analysis])
        features['skill_level'] = game_data['skill_level']
        features['white_elo'] = game_data['white_elo']
        features['black_elo'] = game_data['black_elo']
        features['pgn'] = game_data['pgn']

        all_features.append(features)
    except:
        continue

features_df = pd.DataFrame(all_features)
print(f'\n✓ Features shape: {features_df.shape}')

Features: 100%|██████████| 9000/9000 [00:00<00:00, 9851.76it/s] 


✓ Features shape: (0, 0)


In [11]:
# ==========================================
# FIXED PERFORMANCE-BASED LABELING
# ==========================================

def create_smart_labels(df):
    """
    Label based on ACTUAL performance in each category,
    not just overall Elo rating.

    Handles edge cases like zeros, duplicates, and insufficient data.
    """

    print("🏷️ Creating performance-based labels...\n")

    # 1. OPENING: Based on opening phase CP loss
    print("📖 Opening labels: based on avg_cp_loss_opening")
    try:
        df['label_opening'] = pd.qcut(
            df['avg_cp_loss_opening'],
            q=3,
            labels=['excellent', 'average', 'weak'],
            duplicates='drop'
        )
    except ValueError:
        # Fallback to manual bins if qcut fails
        df['label_opening'] = pd.cut(
            df['avg_cp_loss_opening'],
            bins=[0, 50, 100, 500],
            labels=['excellent', 'average', 'weak']
        )
    print(df['label_opening'].value_counts())

    # 2. MIDDLEGAME: Combined metric (CP loss + blunder rate)
    print("\n⚔️ Middlegame labels: based on CP loss + blunder rate")
    df['_middlegame_score'] = (
        df['avg_cp_loss_middlegame'] +
        (df['blunder_rate_middlegame'] * 200)
    )
    try:
        df['label_middlegame'] = pd.qcut(
            df['_middlegame_score'],
            q=3,
            labels=['excellent', 'average', 'weak'],
            duplicates='drop'
        )
    except ValueError:
        df['label_middlegame'] = pd.cut(
            df['_middlegame_score'],
            bins=[0, 150, 300, 1000],
            labels=['excellent', 'average', 'weak']
        )
    print(df['label_middlegame'].value_counts())

    # 3. ENDGAME: Based on endgame CP loss (SPECIAL HANDLING)
    print("\n🏁 Endgame labels: based on avg_cp_loss_endgame")

    # Check how many games reached endgame
    endgame_games = (df['avg_cp_loss_endgame'] > 0).sum()
    total_games = len(df)
    endgame_pct = endgame_games / total_games * 100

    print(f"   Games that reached endgame: {endgame_games}/{total_games} ({endgame_pct:.1f}%)")

    if endgame_pct < 30:
        # Not enough endgame data - use overall performance as proxy
        print("   ⚠️ Limited endgame data - using overall CP loss as proxy")
        try:
            df['label_endgame'] = pd.qcut(
                df['avg_cp_loss'],
                q=3,
                labels=['excellent', 'average', 'weak'],
                duplicates='drop'
            )
        except ValueError:
            df['label_endgame'] = pd.cut(
                df['avg_cp_loss'],
                bins=[0, 100, 200, 500],
                labels=['excellent', 'average', 'weak']
            )
    else:
        # Only label games that actually reached endgame
        # Filter to only games with endgame moves
        endgame_mask = df['avg_cp_loss_endgame'] > 0

        try:
            # Create labels only for games with endgame
            df.loc[endgame_mask, 'label_endgame'] = pd.qcut(
                df.loc[endgame_mask, 'avg_cp_loss_endgame'],
                q=3,
                labels=['excellent', 'average', 'weak'],
                duplicates='drop'
            )
        except ValueError:
            # Fallback to manual bins
            df.loc[endgame_mask, 'label_endgame'] = pd.cut(
                df.loc[endgame_mask, 'avg_cp_loss_endgame'],
                bins=[0, 80, 150, 500],
                labels=['excellent', 'average', 'weak']
            )

        # For games without endgame, use their overall performance
        no_endgame_mask = ~endgame_mask
        if no_endgame_mask.sum() > 0:
            try:
                df.loc[no_endgame_mask, 'label_endgame'] = pd.qcut(
                    df.loc[no_endgame_mask, 'avg_cp_loss'],
                    q=3,
                    labels=['excellent', 'average', 'weak'],
                    duplicates='drop'
                )
            except ValueError:
                df.loc[no_endgame_mask, 'label_endgame'] = 'average'

    print(df['label_endgame'].value_counts())

    # 4. TACTICS: Based on blunder rate and best move rate
    print("\n🎯 Tactics labels: based on blunder rate - best moves")
    df['_tactics_score'] = (
        df['blunder_rate'] * 100 -
        df['best_moves'] * 50 +
        df['mistake_rate'] * 50
    )
    try:
        df['label_tactics'] = pd.qcut(
            df['_tactics_score'],
            q=3,
            labels=['excellent', 'average', 'weak'],
            duplicates='drop'
        )
    except ValueError:
        df['label_tactics'] = pd.cut(
            df['_tactics_score'],
            bins=[-50, 10, 25, 200],
            labels=['excellent', 'average', 'weak']
        )
    print(df['label_tactics'].value_counts())

    # 5. STRATEGY: Based on overall CP loss
    print("\n♟️ Strategy labels: based on overall avg_cp_loss")
    try:
        df['label_strategy'] = pd.qcut(
            df['avg_cp_loss'],
            q=3,
            labels=['excellent', 'average', 'weak'],
            duplicates='drop'
        )
    except ValueError:
        df['label_strategy'] = pd.cut(
            df['avg_cp_loss'],
            bins=[0, 100, 200, 500],
            labels=['excellent', 'average', 'weak']
        )
    print(df['label_strategy'].value_counts())

    # 6. TIME MANAGEMENT: Based on time pressure moves (if available)
    print("\n⏱️ Time management labels:")

    # Check if we have time data
    has_time_data = (df['avg_move_time'].sum() > 0)

    if has_time_data:
        print("   Using time pressure data")
        try:
            # Lower time pressure = better
            df['label_time_management'] = pd.qcut(
                df['time_pressure_moves'],
                q=3,
                labels=['excellent', 'average', 'weak'],
                duplicates='drop'
            )
        except ValueError:
            # Manual bins if qcut fails
            df['label_time_management'] = pd.cut(
                df['time_pressure_moves'],
                bins=[0, 0.15, 0.35, 1.0],
                labels=['excellent', 'average', 'weak']
            )
    else:
        # No time data - use blunder rate as proxy
        print("   (No time data - using blunder rate as proxy)")
        try:
            df['label_time_management'] = pd.qcut(
                df['blunder_rate'],
                q=3,
                labels=['excellent', 'average', 'weak'],
                duplicates='drop'
            )
        except ValueError:
            df['label_time_management'] = pd.cut(
                df['blunder_rate'],
                bins=[0, 0.10, 0.20, 1.0],
                labels=['excellent', 'average', 'weak']
            )

    print(df['label_time_management'].value_counts())

    # Clean up temporary columns
    df = df.drop(columns=['_middlegame_score', '_tactics_score'], errors='ignore')

    # Final validation
    print("\n" + "="*70)
    print("LABEL VALIDATION")
    print("="*70)

    all_valid = True
    for category in ['opening', 'middlegame', 'endgame', 'tactics', 'strategy', 'time_management']:
        label_col = f'label_{category}'
        n_classes = df[label_col].nunique()
        has_nulls = df[label_col].isna().sum()

        status = "✅" if n_classes == 3 and has_nulls == 0 else "⚠️"
        print(f"{status} {category:20s}: {n_classes} classes, {has_nulls} nulls")

        if n_classes != 3 or has_nulls > 0:
            all_valid = False
            print(f"   Classes: {df[label_col].value_counts().to_dict()}")

    if all_valid:
        print("\n✅ All labels created successfully!")
    else:
        print("\n⚠️ Some labels have issues - check above")

    return df

# Apply the new labeling
features_df = create_smart_labels(features_df)

# Save
features_df.to_csv(dp('training_features.csv'), index=False)
print('\n✓ Saved training_features.csv with smart labels')

🏷️ Creating performance-based labels...

📖 Opening labels: based on avg_cp_loss_opening


KeyError: 'avg_cp_loss_opening'

---
## 🤖 Step 6: Train Models

In [ ]:
# Load if needed
if 'features_df' not in globals():
    features_df = pd.read_csv(dp('training_features.csv'))

# Config
CATEGORIES = ['opening', 'middlegame', 'endgame', 'tactics', 'strategy', 'time_management']

FEATURE_SETS = {
    'opening': ['avg_cp_loss_opening', 'blunder_rate_opening'],
    'middlegame': ['avg_cp_loss_middlegame', 'blunder_rate_middlegame', 'mistake_rate', 'best_moves', 'good_moves'],
    'endgame': ['avg_cp_loss_endgame', 'blunder_rate_endgame', 'inaccuracy_rate'],
    'tactics': ['blunder_rate', 'brilliant_moves', 'best_moves', 'mistake_rate', 'inaccuracy_rate'],
    'strategy': ['avg_cp_loss', 'blunder_rate', 'mistake_rate', 'best_moves', 'good_moves', 'inaccuracy_rate', 'brilliant_moves'],
    'time_management': ['avg_move_time', 'time_pressure_moves', 'blunder_rate', 'mistake_rate', 'avg_cp_loss']
}

MODEL_CONFIG = dict(
    n_estimators=150,
    max_depth=6,
    min_samples_leaf=30,
    min_samples_split=40,
    max_features='sqrt',
    class_weight='balanced',
    random_state=42,
    n_jobs=-1
)

In [17]:
# ─────────────────────────────────────────────────────────────
# STEP 0: RELOAD AND APPLY HYBRID LABELS FRESH
# This ensures we're NOT using stale labels from CSV
# ─────────────────────────────────────────────────────────────

# Reload raw features (WITHOUT any labels)
features_df = pd.read_csv('/content/drive/MyDrive/chess_ml_streams/training_features.csv')

# Drop any existing label columns to avoid stale data
label_cols = [c for c in features_df.columns if c.startswith('label_')]
features_df = features_df.drop(columns=label_cols)

print(f"✅ Loaded {len(features_df)} games, dropped {len(label_cols)} old label columns")
print(f"   Columns remaining: {len(features_df.columns)}")

# ─────────────────────────────────────────────────────────────
# APPLY HYBRID LABELS
# ─────────────────────────────────────────────────────────────

avg_elo = (features_df['white_elo'] + features_df['black_elo']) / 2

def norm(s):
    return s.rank(pct=True)

def make_label(score):
    if score < 0.33:   return 'weak'
    elif score < 0.66: return 'average'
    else:              return 'excellent'

features_df['label_opening'] = (
    norm(avg_elo) * 0.5 +
    (1 - norm(features_df['avg_cp_loss_opening'])) * 0.5
).apply(make_label)

features_df['label_middlegame'] = (
    norm(avg_elo) * 0.5 +
    (1 - norm(features_df['avg_cp_loss_middlegame'])) * 0.5
).apply(make_label)

features_df['label_tactics'] = (
    norm(avg_elo) * 0.4 +
    (1 - norm(features_df['blunder_rate'])) * 0.4 +
    (1 - norm(features_df['mistake_rate'])) * 0.2
).apply(make_label)

features_df['label_strategy'] = (
    norm(avg_elo) * 0.4 +
    (1 - norm(features_df['avg_cp_loss'])) * 0.4 +
    (1 - norm(features_df['inaccuracy_rate'])) * 0.2
).apply(make_label)

features_df['label_time_management'] = (
    norm(avg_elo) * 0.5 +
    (1 - norm(features_df['time_pressure_moves'])) * 0.5
).apply(make_label)

# ENDGAME: Handle games that didn't reach endgame
reached_endgame = features_df['avg_cp_loss_endgame'] > 0
print(f"\n📊 Games that reached endgame: {reached_endgame.sum()} ({reached_endgame.mean()*100:.1f}%)")

endgame_score = (
    norm(avg_elo) * 0.5 +
    (1 - norm(features_df['avg_cp_loss_endgame'])) * 0.5
)
overall_score = (
    norm(avg_elo) * 0.5 +
    (1 - norm(features_df['avg_cp_loss'])) * 0.5
)

features_df['label_endgame'] = np.where(
    reached_endgame,
    endgame_score.apply(make_label),
    overall_score.apply(make_label)
)

# ─────────────────────────────────────────────────────────────
# VERIFY - should NOT be 3000/3000/3000
# ─────────────────────────────────────────────────────────────
print("\n📊 Label distributions (should NOT be perfectly balanced):")
for col in ['label_opening','label_middlegame','label_endgame',
            'label_tactics','label_strategy','label_time_management']:
    print(f"\n{col}:")
    print(features_df[col].value_counts())

✅ Loaded 9000 games, dropped 6 old label columns
   Columns remaining: 19

📊 Games that reached endgame: 3315 (36.8%)

📊 Label distributions (should NOT be perfectly balanced):

label_opening:
label_opening
average      4125
excellent    2503
weak         2372
Name: count, dtype: int64

label_middlegame:
label_middlegame
average      4489
excellent    2344
weak         2167
Name: count, dtype: int64

label_endgame:
label_endgame
average      4598
weak         2961
excellent    1441
Name: count, dtype: int64

label_tactics:
label_tactics
average      5071
excellent    2019
weak         1910
Name: count, dtype: int64

label_strategy:
label_strategy
average      5172
excellent    1948
weak         1880
Name: count, dtype: int64

label_time_management:
label_time_management
average      5677
excellent    1703
weak         1620
Name: count, dtype: int64


In [38]:
# ══════════════════════════════════════════════════════════════
# CELL 1: FINAL LABELS (UNCHANGED)
# ══════════════════════════════════════════════════════════════

import numpy as np
import pandas as pd

features_df = pd.read_csv('/content/drive/MyDrive/chess_ml_streams/training_features.csv')
label_cols = [c for c in features_df.columns if c.startswith('label_')]
features_df = features_df.drop(columns=label_cols)
print(f"✅ Loaded {len(features_df)} games")

avg_elo = (features_df['white_elo'] + features_df['black_elo']) / 2

def norm(s):
    return s.rank(pct=True)

def make_label(score):
    if score < 0.33:   return 'weak'
    elif score < 0.66: return 'average'
    else:              return 'excellent'

features_df['label_opening'] = (
    norm(avg_elo) * 0.5 +
    (1 - norm(features_df['avg_cp_loss_opening'])) * 0.5
).apply(make_label)

features_df['label_middlegame'] = (
    norm(avg_elo) * 0.40 +
    (1 - norm(features_df['avg_cp_loss_middlegame'])) * 0.35 +
    norm(features_df['best_moves']) * 0.15 +
    (1 - norm(features_df['blunder_rate_middlegame'])) * 0.10
).apply(make_label)

features_df['label_tactics'] = (
    norm(avg_elo) * 0.4 +
    (1 - norm(features_df['blunder_rate'])) * 0.4 +
    (1 - norm(features_df['mistake_rate'])) * 0.2
).apply(make_label)

features_df['label_strategy'] = (
    norm(avg_elo) * 0.4 +
    (1 - norm(features_df['avg_cp_loss'])) * 0.4 +
    (1 - norm(features_df['inaccuracy_rate'])) * 0.2
).apply(make_label)

features_df['label_time_management'] = (
    norm(avg_elo) * 0.3 +
    (1 - norm(features_df['time_pressure_moves'])) * 0.3 +
    norm(features_df['avg_move_time']) * 0.2 +
    (1 - norm(features_df['blunder_rate'])) * 0.2
).apply(make_label)

reached_endgame = features_df['avg_cp_loss_endgame'] > 0
no_endgame = ~reached_endgame

features_df['label_endgame'] = 'average'

eg_elo_norm   = avg_elo[reached_endgame].rank(pct=True)
eg_cp_norm    = (1 - features_df.loc[reached_endgame, 'avg_cp_loss_endgame'].rank(pct=True))
eg_blun_norm  = (1 - features_df.loc[reached_endgame, 'blunder_rate_endgame'].rank(pct=True))
eg_score = eg_elo_norm * 0.35 + eg_cp_norm * 0.40 + eg_blun_norm * 0.25
features_df.loc[reached_endgame, 'label_endgame'] = eg_score.apply(make_label).values

neg_elo_norm  = avg_elo[no_endgame].rank(pct=True)
neg_cp_norm   = (1 - features_df.loc[no_endgame, 'avg_cp_loss_middlegame'].rank(pct=True))
neg_blun_norm = (1 - features_df.loc[no_endgame, 'blunder_rate_middlegame'].rank(pct=True))
neg_score = neg_elo_norm * 0.50 + neg_cp_norm * 0.30 + neg_blun_norm * 0.20
features_df.loc[no_endgame, 'label_endgame'] = neg_score.apply(make_label).values

print("Endgame distribution:")
print(features_df['label_endgame'].value_counts())


# ══════════════════════════════════════════════════════════════
# CELL 2: TRAINING (ENDGAME FIXED)
# ══════════════════════════════════════════════════════════════

import os, joblib
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from sklearn.preprocessing import StandardScaler, LabelEncoder
from xgboost import XGBClassifier
from imblearn.over_sampling import SMOTE

def dp(filename):
    return os.path.join('/content/drive/MyDrive/chess_ml_streams', filename)

features_df['reached_endgame_flag'] = (features_df['avg_cp_loss_endgame'] > 0).astype(int)

CATEGORIES = ['opening', 'middlegame', 'endgame', 'tactics', 'strategy', 'time_management']

FEATURE_SETS = {
    'opening':         ['avg_cp_loss_opening', 'blunder_rate_opening'],
    'middlegame':      ['avg_cp_loss_middlegame','best_moves', 'good_moves', 'brilliant_moves', 'inaccuracy_rate'],
    'endgame':         ['avg_cp_loss_endgame', 'blunder_rate_endgame', 'best_moves',
                        'avg_cp_loss_middlegame', 'blunder_rate_middlegame', 'reached_endgame_flag'],
    'tactics':         ['blunder_rate', 'brilliant_moves', 'best_moves', 'mistake_rate', 'good_moves'],
    'strategy':        ['avg_cp_loss', 'blunder_rate', 'mistake_rate', 'best_moves', 'good_moves','brilliant_moves'],
    'time_management': ['avg_move_time', 'time_pressure_moves', 'blunder_rate', 'mistake_rate', 'avg_cp_loss']
}

BASE_CONFIG = dict(
    n_estimators=300,       # fewer trees
    max_depth=3,            # shallower trees
    learning_rate=0.03,
    subsample=0.8,          # slightly more stochastic
    colsample_bytree=0.8,
    reg_alpha=1.0,          # stronger L1
    reg_lambda=2.0,         # stronger L2
    random_state=42,
    n_jobs=-1,
    eval_metric='mlogloss'
)


results = {}

for category in CATEGORIES:
    print(f'\n{"="*70}')
    print(f'Training {category.upper()}')
    print(f'{"="*70}')

    label_col    = f'label_{category}'
    feature_cols = [f for f in FEATURE_SETS[category] if f in features_df.columns]

    X     = features_df[feature_cols]
    y_str = features_df[label_col]

    print("Class distribution:")
    print(y_str.value_counts())

    le = LabelEncoder()
    y  = le.fit_transform(y_str)
    save_name = {'tactics': 'tactical', 'strategy': 'positional'}.get(category, category)
    joblib.dump(le, dp(f'{save_name}_label_encoder.pkl'))

    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.25, random_state=42, stratify=y
    )

    sm = SMOTE(random_state=42)
    X_train_res, y_train_res = sm.fit_resample(X_train, y_train)
    print(f"After SMOTE: {len(y_train_res)} training samples")

    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train_res)
    X_test_scaled  = scaler.transform(X_test)

    model = XGBClassifier(**BASE_CONFIG)
    model.fit(X_train_scaled, y_train_res)

    y_train_pred = model.predict(X_train_scaled)
    y_test_pred  = model.predict(X_test_scaled)

    train_acc = accuracy_score(y_train_res, y_train_pred)
    test_acc  = accuracy_score(y_test, y_test_pred)

    p, r, f1, _ = precision_recall_fscore_support(
        y_test, y_test_pred, average='weighted', zero_division=0
    )

    print(f'\nTrain Acc: {train_acc:.3f} | Test Acc: {test_acc:.3f} | Gap: {train_acc-test_acc:.3f}')
    print(f'Precision: {p:.3f} | Recall: {r:.3f} | F1: {f1:.3f}')

    results[category] = {
        'train_acc': train_acc,
        'test_acc': test_acc,
        'gap': train_acc - test_acc,
        'precision': p,
        'recall': r,
        'f1': f1
    }

    joblib.dump(model,  dp(f'{save_name}_model.pkl'))
    joblib.dump(scaler, dp(f'{save_name}_scaler.pkl'))
    print(f'✓ Saved {save_name}_model.pkl')

# ── Summary ──
print('\n' + '='*80)
print(f"{'Category':<18} {'Train':<8} {'Test':<8} {'Prec':<8} {'Recall':<8} {'F1':<8} {'Status'}")
print('-'*80)

train_sum, test_sum = 0, 0

for cat, m in results.items():
    train_sum += m['train_acc']
    test_sum  += m['test_acc']

    if m['test_acc'] >= 0.60 and m['gap'] <= 0.20:
        status = '✅ Good'
    elif m['test_acc'] >= 0.55:
        status = '⚠️  Acceptable'
    else:
        status = '❌ Low'

    print(f"{cat:<18} {m['train_acc']:<8.3f} {m['test_acc']:<8.3f} "
          f"{m['precision']:<8.3f} {m['recall']:<8.3f} {m['f1']:<8.3f} {status}")

print('='*80)

avg_train = train_sum / len(results)
avg_test  = test_sum  / len(results)

print(f"\n🎯 Average Train Accuracy: {avg_train:.1%}")
print(f"🎯 Average Test Accuracy : {avg_test:.1%}")

good = sum(1 for m in results.values() if m['test_acc'] >= 0.60)
print(f"✅ Categories at 60%+: {good}/6")


✅ Loaded 9000 games
Endgame distribution:
label_endgame
average      4461
excellent    2328
weak         2211
Name: count, dtype: int64

Training OPENING
Class distribution:
label_opening
average      4125
excellent    2503
weak         2372
Name: count, dtype: int64
After SMOTE: 9282 training samples

Train Acc: 0.699 | Test Acc: 0.627 | Gap: 0.072
Precision: 0.626 | Recall: 0.627 | F1: 0.619
✓ Saved opening_model.pkl

Training MIDDLEGAME
Class distribution:
label_middlegame
average      4731
excellent    2197
weak         2072
Name: count, dtype: int64
After SMOTE: 10644 training samples

Train Acc: 0.764 | Test Acc: 0.661 | Gap: 0.103
Precision: 0.677 | Recall: 0.661 | F1: 0.657
✓ Saved middlegame_model.pkl

Training ENDGAME
Class distribution:
label_endgame
average      4461
excellent    2328
weak         2211
Name: count, dtype: int64
After SMOTE: 10038 training samples

Train Acc: 0.720 | Test Acc: 0.607 | Gap: 0.113
Precision: 0.629 | Recall: 0.607 | F1: 0.589
✓ Saved endgame_mo

---
## 📦 Step 7: Download Models

In [39]:
import zipfile

# Create zip
zip_path = '/content/chess_models.zip'
with zipfile.ZipFile(zip_path, 'w') as zipf:
    for f in os.listdir(DRIVE_DIR):
        if f.endswith('.pkl') or f.endswith('.json'):
            zipf.write(dp(f), f)
            print(f'  Added: {f}')

print('\n✓ Created chess_models.zip')
print('📥 Downloading...')
files.download(zip_path)

  Added: categorized_games.json
  Added: game_analyses.json
  Added: model_config.json
  Added: opening_label_encoder.pkl
  Added: opening_model.pkl
  Added: opening_scaler.pkl
  Added: middlegame_label_encoder.pkl
  Added: middlegame_model.pkl
  Added: middlegame_scaler.pkl
  Added: endgame_label_encoder.pkl
  Added: endgame_model.pkl
  Added: endgame_scaler.pkl
  Added: tactical_label_encoder.pkl
  Added: tactical_model.pkl
  Added: tactical_scaler.pkl
  Added: positional_label_encoder.pkl
  Added: positional_model.pkl
  Added: positional_scaler.pkl
  Added: time_management_label_encoder.pkl
  Added: time_management_model.pkl
  Added: time_management_scaler.pkl

✓ Created chess_models.zip
📥 Downloading...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

---
## ✅ SUCCESS!

You've trained chess ML models using **streaming extraction** - downloading only ~100-500MB instead of 30GB!

### Deploy to your backend:
1. Extract `chess_models.zip`
2. Place in `backend/models/`
3. Restart FastAPI server

Your models are now:
- ✅ Properly labeled by Elo rating
- ✅ Free from extreme outliers
- ✅ Compatible with your inference code
- ✅ Actually make logical predictions!

🎉 No more 50% blunder rate = "excellent"! 🎉